In [10]:
import numpy as np
from statsmodels.stats.proportion import proportion_confint
import pandas as pd
from evaluate_model import component_matching_f1

In [11]:
# Читаем сгенерированные запросы
predicted = pd.read_csv("data/predicted_queries_qwen25_8k_1_5B.csv", sep=";", index_col=0)
predicted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ref     962 non-null    object
 1   pred    962 non-null    object
dtypes: object(2)
memory usage: 22.5+ KB


In [12]:
# Читаем запросы, которые были успешно выполнены в тестовой базе 1С
executed = pd.read_csv("data/predicted_queries_executed_qwen25_8k_1_5B.csv", sep=";", index_col=0)
executed.columns = ["ref", "pred"]
executed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 1 to 960
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ref     700 non-null    object
 1   pred    700 non-null    object
dtypes: object(2)
memory usage: 16.4+ KB


In [13]:
# Заполняем признак успешного выполнения запроса
predicted["executed"] = predicted.index.isin(executed.index).astype(int)

In [14]:
# Заполняем признак полного соответствия сгенерированного запроса исходному
predicted["exact"] = predicted.apply(
    lambda row: row["ref"].lower() == row["pred"].lower(),
    axis=1
).astype(int)

In [15]:
# Вычисляем точечную и интервальную оценки Exact matching
n = predicted.shape[0]
k = sum(predicted["exact"])
p_hat = k / n

alpha = 0.05
lower, upper = proportion_confint(count=k, nobs=n, alpha=alpha, method='wilson')

print(f"Точечная оценка Exact matching: {p_hat:.4f}")
print(f"95% CI (Wilson): [{lower:.4f}, {upper:.4f}]")

Точечная оценка Exact matching: 0.2994
95% CI (Wilson): [0.2713, 0.3291]


In [16]:
# Вычисляем точечную и интервальную оценки Execution accuracy
n = predicted.shape[0]
k = sum(predicted["executed"])
p_hat = k / n

alpha = 0.05
lower, upper = proportion_confint(count=k, nobs=n, alpha=alpha, method='wilson')

print(f"Точечная оценка Execution accuracy: {p_hat:.4f}")
print(f"95% CI (Wilson): [{lower:.4f}, {upper:.4f}]")

Точечная оценка Execution accuracy: 0.7277
95% CI (Wilson): [0.6987, 0.7548]


In [17]:
# Для оценки Component matching используем бутстрап
component_scores = predicted.apply(
    lambda row: component_matching_f1(row["pred"], row["ref"]),
    axis=1
)

# Для удобства соберём это в массивы по каждой компоненте
keys = ['SELECT', 'FROM', 'WHERE', 'GROUP BY', 'HAVING', 'ORDER BY']
n = len(component_scores)  # 962

# Превратим в словарь "название компоненты -> список всех F1 по тесту"
all_scores = {key: [] for key in keys}
for sample_dict in component_scores:
    for k in keys:
        all_scores[k].append(sample_dict[k])

# Посчитаем "точечные" средние (как вы уже делаете)
point_estimates = {k: np.mean(all_scores[k]) for k in keys}

# Задаём количество бутстрап-итераций
n_boot = 1000

# Инициализируем структуру для хранения результатов
boot_means = {k: [] for k in keys}

# Запуск бутстрапа
for _ in range(n_boot):
    # Сгенерировать индексы с возвращением
    indices = np.random.choice(range(n), size=n, replace=True)
    
    # Для каждой компоненты посчитать среднее по бутстрап-подвыборке
    for k in keys:
        # Берём F1-значения у нужных индексов
        sampled_scores = [all_scores[k][idx] for idx in indices]
        boot_means[k].append(np.mean(sampled_scores))

# Теперь у нас есть по 1000 средних значений на каждую компоненту
ci_results = {}
for k in keys:
    sorted_vals = np.sort(boot_means[k])
    lower_95 = np.percentile(sorted_vals, 2.5)   # 2.5-й перцентиль
    upper_95 = np.percentile(sorted_vals, 97.5)  # 97.5-й перцентиль
    ci_results[k] = (lower_95, upper_95)

# Печатаем результат
for k in keys:
    print(f"{k} mean F1 = {point_estimates[k]:.4f}, "
          f"95% CI = [{ci_results[k][0]:.4f}, {ci_results[k][1]:.4f}]")


SELECT mean F1 = 0.7777, 95% CI = [0.7542, 0.8001]
FROM mean F1 = 0.7958, 95% CI = [0.7722, 0.8170]
WHERE mean F1 = 0.8659, 95% CI = [0.8484, 0.8801]
GROUP BY mean F1 = 0.9141, 95% CI = [0.8960, 0.9304]
HAVING mean F1 = 0.9883, 95% CI = [0.9818, 0.9943]
ORDER BY mean F1 = 0.9290, 95% CI = [0.9143, 0.9420]
